In [ ]:
import os
os.chdir(r'C:\Users\WeiChuanzheng\Desktop\circos')

In [ ]:
from collections import Counter

In [ ]:
def FastaToDict(file):
    fasta = {}
    with open(file, 'r')as f:
        lines = f.readlines()
    for line in lines:
        if line[0] == '>':
            key = line.strip()
            fasta[key] = []
        else:
            fasta[key].append(line.strip())
    for key,value in list(fasta.items()):
        fasta[key] = ''.join(value)
    return fasta

# GC density

In [ ]:
seq = FastaToDict('Btx623B.chr.fasta')

In [ ]:
with open('chr.len.txt', 'w')as f:
    for k,v in seq.items():
        k = k[1:]
        f.write(f"{k}\t1\t{len(v)}\n")

In [ ]:
#distance:Window size for calculating density
gc_density = []
distance = 1000000
for k,v in seq.items():
    for i in range(0,len(seq[k]),distance):
        if i+distance < len(seq[k]):
            fragment = seq[k][i:i+distance]
            gc_count = (fragment.count('G') + fragment.count('C')) / len(fragment)
#             print(f"{k.lstrip('>')}\t{i+1}\t{i+distance}\t{gc_count}")
            gc_density += [(k[1:],i,i+distance,gc_count)]
        else:
            fragment = seq[k][i:len(seq[k])]
            gc_count = (fragment.count('G') + fragment.count('C')) / len(fragment)
#             print(f"{k.lstrip('>')}\t{i+1}\t{i+distance}\t{gc_count}")
            gc_density += [(k[1:],i,len(seq[k]),gc_count)]

In [ ]:
with open('gc.density.1M.txt', 'w')as f:
    for i in gc_density:
        f.write(f"{i[0]}\t{i[1]}\t{i[2]}\t{i[3]}\n")

# gene density

In [ ]:
def list_to_file(data_2d, filename='output.txt'):
    with open(filename, 'w') as f:
        for row in data_2d:
            f.write('\t'.join(map(str, row)) + '\n')

In [ ]:
ts_pos = pickle.load(open("B_longest_ts_pos.pkl","rb"))

In [ ]:
gene_density = []
for k,v in seq.items():
    k1 = k[1:]
    distance = 1000000
    for i in range(0,len(seq[k]),distance):
        if i+distance < len(seq[k]):
            gene_sets = [ts for ts in ts_pos if ts[1] == k1 and ts[2] >= i and ts[2] < (i+distance)]
#             print(len(gene_sets))
            gene_density += [(k[1:],i+1,i+distance,len(gene_sets))]
        else:
            gene_sets = [ts for ts in ts_pos if ts[1] == k1 and ts[2] >= i and ts[2] < len(seq[k])]
            gene_density += [(k[1:],i+1,len(seq[k]),len(gene_sets))]

In [ ]:
list_to_file(gene_density, filename='gene.density.1M.txt')

# TE density

#### The input file comes from RepeatMasker, and different component information needs to be adjusted according to needs.

In [ ]:
RM_info = []
with open('Btx623.RepeatMasker.out', 'r')as f:
    lines = f.readlines()
for line in lines:
    RM_info += [(line.split())]

In [ ]:
repeat_type = []
for i in RM_info[3:]:
#     if i[10] == 'Simple_repeat':continue
#     if i[10] == 'Satellite':continue
#     if i[10] == 'Low_complexity':continue
#     if i[10] == 'rRNA' :continue
    repeat_type += [(i[4], int(i[5]), int(i[6]), abs(int(i[6])-int(i[5])), i[10].split('/')[0])]
#     repeat_type += [(i[4], int(i[5]), int(i[6]), abs(int(i[6])-int(i[5])), i[10])]

In [ ]:
#View the type and number of repeating elements
Counter(i[4] for i in repeat_type)

In [ ]:
#An example of obtaining a DNA transposon density file
element_type = []
for i in repeat_type:
    if i[4] == 'DNA':
        element_type.append(i)
len(element_type)

In [ ]:
# Number density or length density can be calculaelementd by commenting lines containing '#'
distance = 1000000
element_density = []
for k,v in seq.items():
    k1 = k[1:]
    for i in range(0,len(seq[k]),distance):
        if i+distance < len(seq[k]):
#             element_sets = [ts for ts in el_type if ts[0] == k1 and ts[2] >= i and ts[2] < (i+distance)]
#             element_density += [(k[1:],i+1,i+distance,len(element_sets))]
            element_sets = sum(ts[3] for ts in el_type if ts[0] == k1 and ts[2] >= i and ts[2] < (i+distance))
            element_density += [(k[1:],i+1,i+distance,element_sets/distance)]
#             print(len(element_sets))    
        else:
#             element_sets = [ts for ts in el_type if ts[0] == k1 and ts[2] >= i and ts[2] < len(seq[k])]
#             element_density += [(k[1:],i+1,len(seq[k]),len(element_sets))]
            element_sets = sum(ts[3] for ts in el_type if ts[0] == k1 and ts[2] >= i and ts[2] < (i+distance))
            element_density += [(k[1:],i+1,len(seq[k]),te_sets/(len(seq[k]) - i))]

In [ ]:
list_to_file(element_density, filename='element.density.txt')